In [ ]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("..")
import d2dl_library as d2l

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

100.0%Extracting /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw
111.0%Extracting /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw
100.0%Extracting /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw
159.1%Extracting /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/jovyan/Datasets/FashionMNIST/FashionMNIST/raw
Processing...
Done!
/opt/venv/lib/python3.7/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a te

In [ ]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

print(W.size())


torch.Size([784, 10])


### 实现softmax运算
如何对多维`Tensor`按维度操作：对同一列（`dim=0`）或同一行（`dim=1`）的元素求和，并在结果中保留行和列这两个维度（`keepdim=True`）

In [ ]:
def softmax(X):
    X_exp=X.exp()
    partition=X_exp.sum(dim=1,keepdim=True)
    return X_exp/partition

可以看到，对于随机输入，我们将每个元素变成了非负数，且每一行和为1。

In [ ]:
X=torch.rand((2,5))
X_prob=softmax(X)
print(X_prob,X_prob.sum(dim=1))

tensor([[0.1996, 0.2215, 0.1344, 0.2673, 0.1771],
        [0.1344, 0.2543, 0.1307, 0.2636, 0.2169]]) tensor([1.0000, 1.0000])


### 定义模型
有了`softmax`运算，可以定义softmax回归模型了。这里通过`view`函数将每张原始图像改成长度为`num_input`的向量。

In [ ]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

### 定义损失函数
为了得到标签的预测概率，我们可以使用gather函数。在下面的例子中，变量y_hat是2个样本在3个类别的预测概率，变量y是这2个样本的标签类别。通过使用gather函数，我们得到了2个样本的标签的预测概率。与3.4节（softmax回归）数学表述中标签类别离散值从1开始逐一递增不同，在代码中，标签类别的离散值是从0开始逐一递增的。

In [ ]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat.gather(1,y.view(-1,1)))

### 计算分类准确率
给定一个类别的预测概率分布y_hat，我们把预测概率最大的类别作为输出类别。如果它与真实类别y一致，说明这次预测是正确的。分类准确率即正确预测数量与总预测数量之比。

为了演示准确率的计算，下面定义准确率accuracy函数。其中y_hat.argmax(dim=1)返回矩阵y_hat每行中最大元素的索引，且返回结果与变量y形状相同。相等条件判断式(y_hat.argmax(dim=1) == y)是一个类型为ByteTensor的Tensor，我们用float()将其转换为值为0（相等为假）或1（相等为真）的浮点型Tensor。


In [ ]:
def accuracy(y_hat,y):
    return (y_hat.argmax(dim=1)==y).float().mean().item()

#模型net在数据集data_iter上的准确率
def evaluate_accuracy(data_iter,net):
    acc_sum,n=0.0,0
    for X,y in data_iter:
        acc_sum+=(net(X).argmax(dim=1)==y).float().sum().item()
        n+=y.shape[0]
    return acc_sum/n

#初始准确率应该接近0.1
print(evaluate_accuracy(test_iter,net))

0.1499


In [ ]:
num_epochs,lr=5,0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

AttributeError: module 'd2dl_library' has no attribute 'sgd'